<a href="https://colab.research.google.com/github/moaabid/Plant-Leaf-Disease-Classifcation-Using-Deep-Learning-and-Flutter/blob/main/Plant_Disease_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Plant_Leaf_Disease_Classification using Tensorflow**

Abstract: 

Plant Disease is one of the main threat to global food security. The main loss of food is due to infected plants,which reflexively reduce the production rate. Disease in plant mostly on the leaves affects the quality and quantity of the plant products. The most common diagnosis is primarily performed by examining the plant leaf for the presence of visual symptoms. Lot of us have less knowledge in plant and its disease . So, it's hard to identify disease with our lesser knowledge. 

In this technological era we can easily identify what disease is affected in the plant. This project provide an software solution which automatically classify the plant Disease. Deep learning techniques have been very successful in image classification problems. This project uses Deep Convolutional Neural Network (CNN) to detect plant diseases from images of plant leaves and accurately classify them and small neural network is trained using a publicly available data set of 54000 images, which achieves an accuracy of 98%. This neural network is built using Keras to running on top of the deep learning framework called Tensor Flow. Overall,This project uses deep learning technique to classify the plant leaf disease and by integrating with mobile phone to makes smartphone-assisted disease classification.



# Step 1 : Import Libraries


Below I have imported some libraries.
 

1.   Tensorflow                     
          - Tensorflow is a Machine learning Framework used to Design,Build and Train deep learning Model.Here we imported tensorflow library to do make Computational,Understanding,Discovery,Prediction and Creating models.
2.   numpy
          - Numpy is an python library which provides High performance multidimensional array and tools for working with this array.
3.   matplotlib.pylot
          - Matplotlib library works like an MATLAB.Which helps us to plot Graph, Histogram,Barplot etc,.
4.   Tensorflow Hub
          - TensorFlow Hub is a library for the publication, discovery, and consumption of reusable parts of machine learning models. A module is a self-contained piece of a TensorFlow graph, along with its weights and assets, that can be reused across different tasks in a process known as transfer learning.
5.   Tensorflow_datasets
         - Tensorflow_datasets contains collection of datasets.
6.   layers from Tensorflow keras
         - Layers library availabel in tensorflow keras helps us to create layers in deep learning model.

7.   Logging
         - Logging library helps us to log error messages

In [ ]:
import tensorflow as tf                                                                 #import tensorflow library
!pip install tfds-nightly

In [ ]:
import os

import numpy as np                                                                      #import numpy library
import matplotlib.pyplot as plt                                                         #import matplotlib library 

import tensorflow_hub as hub                                                            #import tensorflow_hub
import tensorflow_datasets as tfds                                                      #import tenorflow_datasets
tfds.disable_progress_bar()

from tensorflow.keras import layers                                                     #import layers from tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

In [ ]:
import logging                              #import logging.
logger = tf.get_logger()                    #Here I'm calling tensorflow logger function and assigned in the logger variable.
logger.setLevel(logging.ERROR)              #Logs message with level ERROR on this logger.

# Step 2 : Download the Plant_leaf_Dataset  using TensorFlow Datasets

In the cell below i download the Diseases affected Plant_leaf dataset using TensorFlow Datasets. (https://www.tensorflow.org/datasets/catalog/plant_village). This dataset is only split into a TRAINING set.Therefore using `tfds.splits` to split this training set into to a `training_set` and a `validation_set` and done a `[70, 30]` split such that 70 corresponds to the `training_set` and 30 to the `validation_set`. Then load the `plant_village` dataset using `tfds.load`. In the `tfds.load` function i uses the parameters such as `split`, `with_info`, `as_supervised`.

In [ ]:
(training_set, validation_set,test_set), dataset_info = tfds.load(
    'plant_village',                                               #loading plant_village_Dataset
    split=['train[80%:]', 'train[80%:90%]', 'train[90%:]'],        #spliting 80 to training set,10% to validation set and 10% test_set.
    with_info=True,                                                #setting with_info True so that we can get dataset information.
    as_supervised=True,                                            #By setting as_supervised value True the tf.data.dataset will return 2 tuples(input,label)
)

# Step 3 : Print Information about the Plant Dataset

Downloading the dataset is completed, Using the dataset info to print the number of classes in the dataset, and also coded to count how many images in the training and validation sets. 

In [ ]:
num_classes = dataset_info.features['label'].num_classes                          #Storing number of classes in num_classes variable.

num_training_examples = 0                                                         #Assign 0 to num_training_examples
num_validation_examples = 0                                                       #Assign 0 to num_validation_examples
num_test_examples = 0

for example in training_set:
  num_training_examples += 1                                                      #Using for loop getting no of training images

for example in validation_set:
  num_validation_examples += 1                                                    #Using for loop getting no of Validation images

for example in test_set:
  num_test_examples += 1  

print('Total Number of Classes: {}'.format(num_classes))                          #printing Total number of classes
print('Total Number of Training Images: {}'.format(num_training_examples))        #printing Total Number of Training Images
print('Total Number of Validation Images: {}'.format(num_validation_examples))    #printing Total Number of Validation Images
print('Total Number of Test Images: {} \n'.format(num_test_examples))

In the below cell checking image shape.

In [ ]:
for i, example in enumerate(training_set.take(5)):
  print('Image {} shape: {} label: {}'.format(i+1, example[0].shape, example[1]))     #Here printing 5 different image shapes and it's corresponding labels.

In [ ]:
image = np.array([[[1], [2]], [[3], [4]]]) 
tf.image.random_flip_up_down(image, None).numpy().tolist() 

# Step 4 : Reformat Images and Create Batches

In the cell below, I create a function that reformats all images to the resolution expected by MobileNet v2 (224, 224) and normalizes them. The function take in an `image` and a `label` as arguments and return the new `image` and corresponding `label`. Then creating training and validation batches of size `32`.

In [ ]:
IMAGE_RES = 224                                                                                                   #Assigning Image resolution as 224

def format_image(image, label):
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0                                                    #In this function i'm resizing images to IMAGE_RES and converting to binary value
  return image, label                                                                                             
                                                                                       #Assigning Batchsize as 32

In [ ]:
BATCH_SIZE = 32 #@param {type:"integer"}


In [ ]:
train_batches = training_set.shuffle(num_training_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)    #Formating and shuffling the images

validation_batches = validation_set.map(format_image).batch(BATCH_SIZE).prefetch(1)                               #Formating validation images
test_batches = test_set.map(format_image).batch(1)


In [ ]:
for i, example in enumerate(train_batches.take(5)):
  print('Image {} shape: {}'.format(i+1, example[0].shape))


In [ ]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape


#  Step 5 : Simple Transfer Learning with TensorFlow Hub

Using TensorFlow Hub to do Transfer Learning.Transfer learning reuse parts of an already trained model and change the final layer, or several layers, of the model, and then retrain those layers on our own dataset.

### Creating a Feature Extractor
In the cell below creating a `feature_extractor` using MobileNet v2.The partial model from TensorFlow Hub (without the final classification layer) is called a feature vector.[TensorFlow Hub documentation](https://tfhub.dev/s?module-type=image-feature-vector&q=tf2) to see list of available feature vectors and Finally, creating a `feature_extractor` by using `hub.KerasLayer` with the correct `input_shape` parameter.

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"  #mobilenet v2 url without final classfication layer
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES, 3))   #Extracting layers from the url with corresponding Image resolution

### Freezing the Pre-Trained Model

In the cell below freezing the variables in the feature extractor layer, so that the training only modifies the final classifier layer.

In [ ]:
feature_extractor.trainable = False

### Attaching a classification head

In the cell below creating a `tf.keras.Sequential` model, and adding the pre-trained model and the new classification layer.The classification layer must have the same number of classes as plant dataset. Finally printing a summary of the Sequential model.

In [ ]:
model = tf.keras.Sequential([
  feature_extractor,                          #pre-trained model
  layers.Dense(num_classes)                   #new classifcation Dense layers with number of classes in the plant dataset
])

model.summary()                               #summary of the sequential model

# Step 6 : Train the model

In the cell bellow training the model, by first calling `compile` and then followed by `fit`. Training the model for only 8 epochs.

In [ ]:
model.compile(
  optimizer='adam',                                                             #Using adam as an optimizer
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),         #Using SparseCategoricalCrossentropy to calculate loss
  metrics=['accuracy'])

EPOCHS = 5                                                                      #Assigning epochs as 5

history = model.fit(train_batches,                                              #Here i'm training train dataset with epochs 10 and validating with validation dataset
                    epochs=EPOCHS,
                    validation_data=validation_batches)

# Step 7 : Plot Training and Validation Graphs

In the cell below, plotting the training and validation accuracy/loss graphs.

In [ ]:
acc = history.history['accuracy']                                                 #Getting accuracy history
val_acc = history.history['val_accuracy']                                         #Getting validation accuracy history

loss = history.history['loss']                                                    #Getting loss history
val_loss = history.history['val_loss']                                            #Gettting validaiton loss history 

epochs_range = range(EPOCHS)                                                      #Getting Epochs range

plt.figure(figsize=(8, 8))                                                        #figure size width=8 and height=8
plt.subplot(1, 2, 1)                                                              #subplot Row=1 Column=2 
plt.plot(epochs_range, acc, label='Training Accuracy')                            #ploting Training accuracy
plt.plot(epochs_range, val_acc, label='Validation Accuracy')                      #ploting validation accuracy
plt.legend(loc='lower right')                                                     #label in lower right
plt.title('Training and Validation Accuracy')                                     #Title

plt.subplot(1, 2, 2)                                                              #subplot Row=1 Column=2
plt.plot(epochs_range, loss, label='Training Loss')                               #ploting Training loss
plt.plot(epochs_range, val_loss, label='Validation Loss')                         #ploting Validation loss
plt.legend(loc='upper right')                                                     #label in upper right
plt.title('Training and Validation Loss')                                         #Title
plt.show()                                                                        #show function used to show figure

# Step 8 : Check Predictions

In the cell below getting the label names from the dataset info and convert them into a NumPy array. Printing the array to make sure the correct label names.

In [ ]:
class_names = np.array(dataset_info.features['label'].names)              #Getting label names from dataset info and converted to numpy

print(class_names)

### Creating an Image Batch and Make Predictions

In the cell below, Using the `next()` function to create an `image_batch` and its corresponding `label_batch`. Converting both the `image_batch` and `label_batch` to numpy arrays using the `.numpy()` method. Then using the `.predict()` method to run the image batch through the model and make predictions. Then using the `np.argmax()` function to get the indices of the best prediction for each image. Finally converting the indices of the best predictions to class names.

In [ ]:
#creating image batch and label batch using next function
image_batch, label_batch = next(iter(train_batches))         

#converting image batch to numoy array using numpy function
image_batch = image_batch.numpy()                              

#converting label batch to numoy array using numpy function
label_batch = label_batch.numpy()

#Using predict function and run the image batch through the model and make predictions
predicted_batch = model.predict(image_batch)
predicted_batch = tf.squeeze(predicted_batch).numpy()

#Using mp.argmax function to get indices of the best prediction for each image
predicted_ids = np.argmax(predicted_batch, axis=-1)

#converting indices to class names
predicted_class_names = class_names[predicted_ids]

print(predicted_class_names)

### Printing True Labels and Predicted Indices

In the cell below, printing the true labels and the indices of predicted labels.

In [ ]:
print("Labels:           ", label_batch)          
print("Predicted labels: ", predicted_ids)

# Step 9 : Plot Model Predictions

In [ ]:
plt.figure(figsize=(10,9))                                                    #figure width=10 Height =9   

for n in range(10):                                                           #Here plotting 10 predicted model
  plt.subplot(5,2,n+1)  
  plt.subplots_adjust(hspace = .3)
  plt.imshow(image_batch[n])
  color = "White" if predicted_ids[n] == label_batch[n] else "red"
  plt.title(predicted_class_names[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (blue: correct, red: incorrect)")

# Step 10 : Export Model



In [ ]:
PLANT_DISEASE_SAVED_MODEL = "exp_saved_model"

In [ ]:
tf.saved_model.save(model, PLANT_DISEASE_SAVED_MODEL)

In [ ]:
%%bash -s $PLANT_DISEASE_SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

In [ ]:
loaded = tf.saved_model.load(PLANT_DISEASE_SAVED_MODEL)

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

# Step 11:  Convert using TFLite's Converter


Loading the TFLiteConverter with the SavedModel

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(PLANT_DISEASE_SAVED_MODEL)

### Post-training quantization
The simplest form of post-training quantization quantizes weights from floating point to 8-bits of precision. This technique is enabled as an option in the TensorFlow Lite converter. At inference, weights are converted from 8-bits of precision to floating point and computed using floating-point kernels. This conversion is done once and cached to reduce latency.

To further improve latency, hybrid operators dynamically quantize activations to 8-bits and perform computations with 8-bit weights and activations. This optimization provides latencies close to fully fixed-point inference. However, the outputs are still stored using floating point, so that the speedup with hybrid ops is less than a full fixed-point computation.

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

### Post-training integer quantization
We can get further latency improvements, reductions in peak memory usage, and access to integer only hardware accelerators by making sure all model math is quantized. To do this, we need to measure the dynamic range of activations and inputs with a representative data set. You can simply create an input data generator and provide it to our converter.

In [ ]:
def representative_data_gen():
  for input_value, _ in test_batches.take(100):
    yield [input_value]

In [ ]:
converter.representative_dataset = representative_data_gen

The resulting model will be fully quantized but still take float input and output for convenience.

Ops that do not have quantized implementations will automatically be left in floating point. This allows conversion to occur smoothly but may restrict deployment to accelerators that support float. 

In [ ]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

### Finally convert the model

In [ ]:
tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
  f.write(tflite_model)

# Step 12 :Test the TFLite model using the Python Interpreter

In [ ]:
# Load TFLite model and allocate tensors.
  
interpreter = tf.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

In [ ]:
from tqdm import tqdm

# Gather results for the randomly sampled test images
predictions = []

test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(10)):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

In [ ]:
#@title Utility functions for plotting
# Utilities for plotting

# class_names = class_names

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
    
  img = np.squeeze(img)

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'green'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

In [ ]:
#@title Visualize the outputs { run: "auto" }
index = 4 #@param {type:"slider", min:0, max:9, step:1}
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(index, predictions, test_labels, test_imgs)
plt.show()

Download the model.


In [ ]:

labels = class_names
with open('labels.txt', 'w') as f:
  f.write('\n'.join(labels))

try:
  from google.colab import files
  files.download('converted_model.tflite')
  files.download('labels.txt')
except:
  pass